# Análisis y Limpieza de Datos -- Procedimiento para Catalogar Cáncer Luminal A y B

Este archivo contiene el proceso de limpieza, exploración y análisis de la base de datos del proyecto de de cáncer de mama en base a la información de los diagnosticos de diversos pacientes registrados por la Secretaria de Salud.

mas info sobre lo q buscamos


---

In [74]:
import pandas as pd

#Leemos la Base de Datos
df = pd.read_csv('./resources/2025_08_27 BD Proy Salud.csv', encoding='latin1')
pd.set_option('display.max_columns', None)

#hay dos Er_%, uno con _ y oyto con espacio, lo mismo con Pr_%, elimina los que tienen espacios
df = df.drop(columns=['ER %', 'PR %'])


#Renombramos columnas para agregar _ entre espacios

df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.title()

#Sacamos columnas disponibles para facilidad de lectura posterior
unique_columns = df.columns.tolist()

print("Numero de columnas/variables:", len(unique_columns), "\n")
print("Columnas identificadas: \n", unique_columns)




Numero de columnas/variables: 109 

Columnas identificadas: 
 ['Id', 'Fecha_De_Registro', 'Fecha_De_Nacimiento', 'Fecha_De_Dx_Biopsia', 'Edad_Al_Dx_(Años)', 'Fecha_1°_Consulta_Om_Cecan', 'Peso_Al_Dx', 'Talla_Al_Dx', 'Imc', 'Edo_Menopausia_Dx', 'Preservación_De_La_Fertilidad', 'Prueba_Genetica', 'Resultado_Panel_Genético', 'Otra_Mutación', 'Significado', 'Cm_Bilateral', 'Cm_Asociado_Al_Embarazo', 'Ec', 'T', 'N', 'M', 'Tipo_Histológico', 'Otro', 'Grado', 'Er', 'Er_%', 'Pr', 'Pr_%', 'Her2_/_Neu', 'Her2_Ihq_+++', 'Fish/Sish', 'Ki67', 'Ki67_%', 'Ilv', 'Tils', 'Tils_%', 'Patología_Rev_Cecan', 'Tx_Neoadyuvante', 'Tx_Neoadyuvante_Cecan', 'Quimioterapia_Neoadyuvante', 'Esquema_Qt_Neoadyuvante', 'Antiher2_Neoadyuvante', 'Esquema_Antiher2_Neoadyuvante', 'Inmunoterapia_Neoadyuvante', 'Comentarios_Tx_Neoadyuvante', 'Respuesta_Patológica_Completa', 'Ypt', 'Ypn', 'Rbc', 'Fecha_De_Cx', 'Cx_Cecan', 'Tipo_Cx_Mama', 'Manejo_Axila', 'Requirio_2°_Cx_De_Mama', 'Cual', 'Rt_Adyuvante', 'Tipo_Rt_Ady', 'Analógo

En esta acción inicial: 
- Modificamos los nombres de las columnas para manipularlas de manera más sencilla
- Modificamos nombres similares para las variables de recurrencia de PR y ER para no ser confundidas con el porcentaje de PR y ER
- Retornamos una lista de todas las columnas para identificar mejoras, errores, y verificar cuales no son más utiles de una manera más eficiente


---
##### Creamos una sub-tabla con las columnas que corresponden a el objetivo de nuestro análisis

Seleccionamos datos de interés para el análisis de cáncer luminal, además de datos demográficos básicos


In [75]:
#Seleccionamos datos de interés para el análisis de cáncer luminal, además de datos demográficos básicos

luminal_columns = [
    'Id', 'Fecha_De_Registro', 'Fecha_De_Nacimiento', 'Fecha_De_Dx_Biopsia', 'Edad_Al_Dx_(Años)',
    'Fecha_1°_Consulta_Om_Cecan', 'Peso_Al_Dx', 'Talla_Al_Dx', 'Er_%', 'Pr_%', 'Her2_/_Neu',
    'Her2_Ihq_+++','Fish/Sish', 'Ki67_%', 'Grado'
]

#Nuevo DataFrame con columnas de interés
luminalDf = df[luminal_columns]

#Mostramos todas las columnas y los primeros registros
pd.set_option('display.max_columns', None)
luminalDf.head()


print("Cantidad de registros:", luminalDf.shape[0])



Cantidad de registros: 1366


# Antes de comenzar...

Previo a cualquier análisis, necesitamos limpiar repeticiones, datos redundantes entre otros errores


In [76]:
#Observamos la cantidad de registros que tenemos

print("Cantidad de registros:", luminalDf.shape[0])


Cantidad de registros: 1366


In [77]:
#Limpiamos IDs repetidos y nulos, nos quedamos con los primeros repetidos

luminalDf = luminalDf.drop_duplicates(subset=['Id'], keep='first')
luminalDf = luminalDf[luminalDf['Id'].notna()]

#Mostramos la cantidad de registros después de la limpieza
print("Cantidad de registros después de la limpieza:", luminalDf.shape[0])

Cantidad de registros después de la limpieza: 1351


Al menos 15 registros estaban duplicados o nulos

---
##### Analizamos los tipos de datos de las columnas

In [78]:

pd.set_option('display.max_rows', None)

print("Tipos de datos de cada columna:")
print(luminalDf.dtypes)




Tipos de datos de cada columna:
Id                             object
Fecha_De_Registro              object
Fecha_De_Nacimiento            object
Fecha_De_Dx_Biopsia            object
Edad_Al_Dx_(Años)               int64
Fecha_1°_Consulta_Om_Cecan     object
Peso_Al_Dx                      int64
Talla_Al_Dx                   float64
Er_%                           object
Pr_%                           object
Her2_/_Neu                     object
Her2_Ihq_+++                   object
Fish/Sish                      object
Ki67_%                         object
Grado                          object
dtype: object


##### Podemos identificar varios tipos de datos que no son correctos

- Podemos convertir las fechas a datetime.
- Los valores de Er_% y Pr_% deben ser convertidos a enteros.



In [79]:

#Cambiamos fechas a datetime
for col in luminalDf.columns:
    if 'Fecha' in col:
        luminalDf[col] = pd.to_datetime(luminalDf[col], errors='coerce') 

columnasFechas = [col for col in luminalDf.columns if 'Fecha' in col]


print(luminalDf[columnasFechas].dtypes)

Fecha_De_Registro             datetime64[ns]
Fecha_De_Nacimiento           datetime64[ns]
Fecha_De_Dx_Biopsia           datetime64[ns]
Fecha_1°_Consulta_Om_Cecan    datetime64[ns]
dtype: object


In [80]:

luminalDf = luminalDf.copy()  
luminalDf['Er_%'] = pd.to_numeric(luminalDf['Er_%'], errors='coerce').astype('Int64')
luminalDf['Pr_%'] = pd.to_numeric(luminalDf['Pr_%'], errors='coerce').astype('Int64')
luminalDf['Grado'] = pd.to_numeric(luminalDf['Grado'], errors='coerce').astype('Int64')
luminalDf['Ki67_%'] = pd.to_numeric(luminalDf['Ki67_%'], errors='coerce').astype('Int64')


print(luminalDf[['Er_%', 'Pr_%', 'Grado', 'Ki67_%']].dtypes)



Er_%      Int64
Pr_%      Int64
Grado     Int64
Ki67_%    Int64
dtype: object


---
Una vez que corregimos los tipos de datos, verificamos que si estén correctos


In [81]:
print(luminalDf.dtypes)

Id                                    object
Fecha_De_Registro             datetime64[ns]
Fecha_De_Nacimiento           datetime64[ns]
Fecha_De_Dx_Biopsia           datetime64[ns]
Edad_Al_Dx_(Años)                      int64
Fecha_1°_Consulta_Om_Cecan    datetime64[ns]
Peso_Al_Dx                             int64
Talla_Al_Dx                          float64
Er_%                                   Int64
Pr_%                                   Int64
Her2_/_Neu                            object
Her2_Ihq_+++                          object
Fish/Sish                             object
Ki67_%                                 Int64
Grado                                  Int64
dtype: object


##### Buscamos datos únicos para cada columna 

Identificamos datos únicos para observar irregularidades, duplicados, valores 

In [82]:
#Obtenemos datos únicos para cada columna para estas columnas 'Er_%', 'Pr_%', 'Her2_/_Neu', que añada cada columna a un csv independiente

try:
    for col in ['Er_%', 'Pr_%', 'Her2_/_Neu', 'Her2_Ihq_+++','Fish/Sish', 'Ki67_%', 'Grado']:
        if col in luminalDf.columns:
            print(f'Columna: {col}\nValores únicos:\n{luminalDf[col].unique()}\n')
        else:
            print(f'Columna {col} no encontrada en luminalDf\n')
            
except Exception as e:
    pass


Columna: Er_%
Valores únicos:
<IntegerArray>
[  10, <NA>,    0,   80,  100,   15,   20,   90,    2,   85,   70,   12,   98,
   60,    5,   95,   30,   50,    4,   65,   40,    7,    1,   55,    8,    6,
   75,    3,   91,   25,   96,   35,   99,    9,   16]
Length: 35, dtype: Int64

Columna: Pr_%
Valores únicos:
<IntegerArray>
[ 100,    5, <NA>,   60,   90,    0,   95,   40,   80,    2,   20,   70,   10,
    3,   50,   25,   30,    8,    7,   35,   65,   75,   15,    1,   18,   22,
   12,    4,   85,   99,   98,    6,   88,   92,   68,    9]
Length: 36, dtype: Int64

Columna: Her2_/_Neu
Valores únicos:
['Negativo ' 'Negativo' 'No aplica' 'Positivo' 'Desconocido']

Columna: Her2_Ihq_+++
Valores únicos:
['1+' 'No aplica' '0+' '2+' '3+' 'Desconocido']

Columna: Fish/Sish
Valores únicos:
['No aplica' 'Negativo' 'Positivo' 'Desconocido']

Columna: Ki67_%
Valores únicos:
<IntegerArray>
[   5, <NA>,   20,    0,   90,   10,   50,   40,   60,    8,   65,   80,   95,
   25,   30,   70,   15,   4

---
##### Hacemos la limpieza de los datos, esto incluyo:

- Reemplazar las filas numéricas con valores de "No Aplica" y "Desconocido" por NaN, con la justificación
  de que no cuentan con valores para catalogar en Luminal A y Luminal B

In [83]:
#Borrar todas las filas que tengan en sus celdas de las columnas categóricas los valores 'No Aplica' o 'Desconocido'

import numpy as np
categorical_cols = ['Her2_/_Neu', 'Her2_Ihq_+++', 'Fish/Sish', 'Grado']  
# Reemplazar 'No Aplica' y 'Desconocido' por np.nan en las columnas categóricas

#Eliminar datos duplicados en las columnas categóricas, usar strip para verificar duplicidad

luminalDf['Her2_/_Neu'] = luminalDf['Her2_/_Neu'].str.strip()
luminalDf['Her2_Ihq_+++'] = luminalDf['Her2_Ihq_+++'].str.strip()
luminalDf['Fish/Sish'] = luminalDf['Fish/Sish'].str.strip()



for col in categorical_cols:
    luminalDf[col] = luminalDf[col].replace(['No Aplica', 'No aplica', 'Desconocido', 'desconocido'], np.nan)




try:
    for col in ['Er_%', 'Pr_%', 'Her2_/_Neu', 'Her2_Ihq_+++','Fish/Sish', 'Ki67_%', 'Grado']:
        if col in luminalDf.columns:
            print(f'Columna: {col}\nValores únicos:\n{luminalDf[col].unique()}\n')
            luminalDf[col].to_csv(f'./resources/valores_unicos_{col}.csv', index=False)

        else:
            print(f'Columna {col} no encontrada en luminalDf\n')
            
except Exception as e:
    pass

Columna: Er_%
Valores únicos:
<IntegerArray>
[  10, <NA>,    0,   80,  100,   15,   20,   90,    2,   85,   70,   12,   98,
   60,    5,   95,   30,   50,    4,   65,   40,    7,    1,   55,    8,    6,
   75,    3,   91,   25,   96,   35,   99,    9,   16]
Length: 35, dtype: Int64

Columna: Pr_%
Valores únicos:
<IntegerArray>
[ 100,    5, <NA>,   60,   90,    0,   95,   40,   80,    2,   20,   70,   10,
    3,   50,   25,   30,    8,    7,   35,   65,   75,   15,    1,   18,   22,
   12,    4,   85,   99,   98,    6,   88,   92,   68,    9]
Length: 36, dtype: Int64

Columna: Her2_/_Neu
Valores únicos:
['Negativo' nan 'Positivo']



In [84]:
luminalDf.tail(15)

,Id,Fecha_De_Registro,Fecha_De_Nacimiento,Fecha_De_Dx_Biopsia,Edad_Al_Dx_(Años),Fecha_1°_Consulta_Om_Cecan,Peso_Al_Dx,Talla_Al_Dx,Er_%,Pr_%,Her2_/_Neu,Her2_Ihq_+++,Fish/Sish,Ki67_%,Grado
1351,5448-17,2017-11-22,1966-12-22,2017-11-14,50,2017-11-22,81,1.68,100,3,Negativo,0+,NaN,60,3
1352,5477-17,2017-11-27,1957-08-10,2017-11-22,60,2017-12-06,75,1.60,0,0,Negativo,0+,NaN,80,2
1353,5479-17,2017-05-24,1964-05-19,2017-11-29,53,2017-12-06,74,1.62,100,10,Negativo,0+,NaN,35,3
1354,5482-17,2017-09-19,1973-02-26,2017-10-01,44,2017-11-29,73,1.66,80,0,Positivo,3+,NaN,20,3
1355,5485-17,2017-06-27,1966-12-17,2017-07-19,50,2017-12-27,67,1.56,0,0,Positivo,3+,NaN,<NA>,2
1356,5486-17,2017-11-29,1976-02-03,2018-01-16,41,2018-01-08,55,1.47,95,90,Negativo,2+,Negativo,<NA>,1
1357,5487-17,2017-07-26,1964-01-06,2018-02-09,54,2018-01-09,66,1.58,0,0,Negativo,0+,NaN,<NA>,3
1358,5497-17,2016-08-16,1955-11-20,2018-02-12,62,2017-12-20,76,1.60,100,100,Negativo,0+,NaN,5,2
1359,5512-17,2017-03-11,1959-08-05,2017-11-23,58,2017-12-12,63,1.60,100,0,Positivo,3+,NaN,30,2
1360,5561-17,2017-07-01,1960-05-01,2018-05-05,58,2018-02-19,60,1.60,0,0,Negativo,0+,NaN,30,3


In [85]:
print(luminalDf.isna().sum())  # Nulos por columna
print(luminalDf.isna().sum(axis=1).value_counts()) 


print("Cantidad de registros:", luminalDf.shape[0])


Id                               0
Fecha_De_Registro                0
Fecha_De_Nacimiento              1
Fecha_De_Dx_Biopsia              1
Edad_Al_Dx_(Años)                0
Fecha_1°_Consulta_Om_Cecan       3
Peso_Al_Dx                       0
Talla_Al_Dx                      0
Er_%                            30
Pr_%                            31
Her2_/_Neu                      31
Her2_Ihq_+++                    29
Fish/Sish                     1253
Ki67_%                         501
Grado                           15
dtype: int64
1    818
2    435
0     61
7     13
6     11
3      5
4      5
5      3
Name: count, dtype: int64
Cantidad de registros: 1351
